# Traditional machine learning models for age prediction on EEG data

This notebook uses traditional ML methods to predict the age of infants using EEG data. The EEG data is preprocessed.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from eegyolk.config import Config
from eegyolk.loaders import RegressionsLoader
from eegyolk.ml import Regressions
from eegyolk.nn import NnOptimizer

config = Config()

## Load preprocessed data

Steps:

1. Get all the files in the output folder
2. Get the full paths of the files without the .h5 or .csv extensions
3. Load the features from the .h5 files
4. Assign the proper labels to the files based on the metadata
5. Assign the subject's code to the files based on the metadata
6. Split the data into a training, validation and test set (NOTE: make sure data points from same subjects don't end up in same set

In [ ]:
def plot_result(df, prop, x, y):
    sns.set()
    ax = sns.scatterplot(x=x, y=y, hue=prop, palette='RdBu', data=df)

    norm = plt.Normalize(df[prop].min(), df[prop].max())
    sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
    sm.set_array([])

    # Remove the legend and add a colorbar
    ax.get_legend().remove()
    ax.figure.colorbar(sm)

    plt.show()

In [ ]:
%%time
rloader = RegressionsLoader(config.get_directory('preprocessed'), config.get_directory('models'), samples=100)
rloader.load()
rloader.split()
regressions = Regressions(rloader)

## Dummy regressor

Firstly, we make predictions with dummy regressors as a simple baseline to see whether other models learn "something". From the sklearn docs: 

> `DummyRegressor` is a regressor that makes predictions using simple rules. This regressor is useful as a simple baseline to compare with other (real) regressors."

In [ ]:
%%time
regressions.algorithms['dummy'].fit()

## Model 1: Random Forest regressor

In [ ]:
%%time
regressions.algorithms['rf'].grid_search()

### Train with best parameters

In [ ]:
%%time
regressions.algorithms['rf'].fit()

## Model 2: Linear Support Vector Regressor

There are a lot of training examples in the training set. According to the sklearn docs: "The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to datasets with more than a couple of 10000 samples." 

They recommend using a linear SVR for large data sets. Therefore, let's try this first.

### Randomized search

In [ ]:
%%time
lsv_result = regressions.algorithms['lsv'].fit()

In [ ]:
cv_df = pd.DataFrame(lsv_result.cv_results_).sort_values('rank_test_score').head(50)
cv_df.head(30)

In [ ]:
plot_result(cv_df, 'mean_test_score', "param_linearsvr__C", "param_linearsvr__epsilon")

### Grid search

In [ ]:
%%time
lsv_gs_result = regressions.algorithms['lsv'].grid_search()

In [ ]:
lsv_gs_result.best_params_

In [ ]:
cv_df_gs = pd.DataFrame(lsv_gs_result.cv_results_).sort_values('rank_test_score').head(50)
cv_df_gs.head(30)

In [ ]:
plot_result(cv_df_gs, 'mean_test_score', "param_linearsvr__C", "param_linearsvr__epsilon")

### Train on all data with best parameters

In [ ]:
%%time
regressions.algorithms['lsv'].best_fit()

## Model 3: (Non-linear) Support Vector Regressor

Let's try fitting a SVR on a (small) chunk of the training data. The parameter search below is quite small, but a broader search has been done before. However, a more fine-grained search is still necessary. The downside of SVR with a non-linear kernel is that it's very slow to fit and predict.

### Randomized search

In [ ]:
%%time
nl_srv_result = regressions.algorithms['svr'].fit()

In [ ]:
nl_srv_result.best_params_

In [ ]:
df_rs_svr = pd.DataFrame(nl_srv_result.cv_results_).sort_values('rank_test_score')
df_rs_svr = df_rs_svr.loc[df_rs_svr['param_svr__gamma'] < 0.0025].head(20)
df_rs_svr.head(30)

In [ ]:
plot_result(df_rs_svr, 'mean_test_score', 'param_svr__C', 'param_svr__epsilon')

In [ ]:
sns.scatterplot(x="param_svr__gamma", y="mean_test_score", data=df_rs_svr)

### Grid search

In [ ]:
%%time
svr_gs_result = regressions.algorithms['svr'].grid_search()

In [ ]:
svr_gs_result.best_params_

In [ ]:
df_gs_svr = pd.DataFrame(svr_gs_result.cv_results_).sort_values('rank_test_score')
df_gs_svr.head(30)

In [ ]:
plot_result(df_gs_svr, 'mean_test_score', 'param_svr__C', 'param_svr__epsilon')

### Train with best parameters

In [ ]:
%%time
svr_result = regressions.algorithms['svr'].best_fit()

## Model 4: SGD Regressor

Inschatting tijd, mijn computer:
    
- X min voor een SGD (1 configuratie)
- RandomizedSearch: 250 iteraties, 5 folds per iteratie = 1250
- 1250 SGD * X = X uur (Schatting met mijn 1 core)

Memory usage:
- X GB per core?

Fitting a SVR is computationally expensive. Therefore, we try prediction with an SGD Regressor. According to the sklearn documentation, it's best to start with a RandomizedSearchCV to find reasonable hyperparameters. Therefore, we start with this.

### Randomized search

In [ ]:
%%time
sgd_result = regressions.algorithms['sgd'].fit()

In [ ]:
sgd_result.best_params_

In [ ]:
df_rs_sgd = pd.DataFrame(sgd_result.cv_results_).sort_values('rank_test_score')
df_rs_sgd = df_rs_sgd.loc[
    df_rs_sgd['param_sgdregressor__loss'] == 'huber'
].sort_values('rank_test_score').head(5000)
df_rs_sgd.head(60)

In [ ]:
sns.scatterplot(x="param_sgdregressor__alpha", y="mean_test_score", data=df_rs_sgd)

### Grid search

In [ ]:
%%time
sgd_gs_result = regressions.algorithms['sgd'].grid_search()

In [ ]:
sgd_gs_result.best_params_

In [ ]:
df_gs_sgd = pd.DataFrame(sgd_gs_result.cv_results_).sort_values('rank_test_score')
df_gs_sgd.head()

In [ ]:
plot_result(df_gs_sgd, 'mean_test_score', 'param_sgdregressor__alpha', 'param_sgdregressor__epsilon')

### Train with best parameters

In [ ]:
%%time
regressions.algorithms['sgd'].best_fit()

## Model 5: Relevance Vector Regression

An alternative to the SVR is the Relevance Vector Machine, also used by Vandenbosch (2018). This isn't included in sklearn, but there are two packages called 'scikit-rvm' and 'sklearn-rvm' using the sklearn API that has implemented this.

### Randomized search

Inschatting tijd, mijn computer:
    
- 4 min voor een RVR (1 configuratie)
- RandomizedSearch: 250 iteraties, 5 folds per iteratie = 1250
- 1250 RVR * 4 min = 84 uur (Schatting met mijn 2 cores)

Memory usage:
- 4 GB per core?

In [ ]:
%%time
emrvr_result = regressions.algorithms['emrvr'].fit()

In [ ]:
emrvr_result.best_params_

### Train on best SVR parameters

In [ ]:
%%time
regressions.algorithms['emrvr'].best_fit()

### Grid search

Inschatting tijd, mijn computer: 

- 4 min voor 1 RVR (1 configuratie). 
- GridSearch: 50 configuraties, 5 folds per configuratie = 250
- 250 RVR * 4 min = 17 uur (Schatting met mijn 2 cores)

<div class="alert alert-block alert-warning">
    TODO(wvxvw): The code below seems bogus.
    The pipeline uses unexpected kernel.
    I don't know why it does this. Need more info
</div>

In [ ]:
from sklearn_rvm import EMRVR

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

parameters = {'svr__epsilon': [4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8],
              'svr__gamma': ['scale', 'auto', 0.0015]
}

pipe  = make_pipeline(StandardScaler(),
                      SVR(verbose=True, kernel='rbf'))

RVR_gridsearch = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=10)

RVR_gridsearch.fit(chunked_X_train[0], chunked_y_train[0])

output_file = os.path.join(PATH_MODELS, 'RVR_gridsearch.joblib')
dump(RVR_gridsearch, output_file)

In [ ]:
RVR_gridsearch.best_params_

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

try:
    RVR_gridsearch
except:
    RVR_gridsearch = load(os.path.join(PATH_MODELS, 'RVR_gridsearch.joblib'))    

# Update verbosity
RVR_gridsearch.verbose = 0

# R2
score = RVR_gridsearch.score(X_test, y_test)

# MSE
predictions = RVR_gridsearch.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Performance of Relevance Vector Regressor: R-squared = {score}, RMSE = {rmse} and MAE = {mae}.")

del rvr_reg

## Model 6: Neural network

In [ ]:
optimizer = NnOptimizer(rloader, epochs=20)

### Plot NN training history

In [ ]:
def plot_loss(history):
    """ Plots the MSE, RMSE, and MAE loss for the training and validation data over time """
    
    %matplotlib inline
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(12, 12), dpi=200)

    ax1.plot(history.history['loss'], label='training data')  
    min_loss = min(history.history['val_loss'])
    val_plot1 = ax1.plot(history.history['val_loss'], label='validation data')
    ax1.axhline(y = min_loss, color = val_plot1[0].get_color(), linestyle='--') 
    x0,x1 = ax1.get_xlim()
    ax1.text(x1, min_loss, "{:.2f}".format(min_loss), ha='left', va='center')
    ax1.set_title('MSE loss')
    ax1.set_ylabel('MSE')
    ax1.set_xlabel('epochs')
    ax1.legend()

    ax2.plot(history.history['root_mean_squared_error'], label='training data')
    min_loss = min(history.history['val_root_mean_squared_error'])
    val_plot2 = ax2.plot(history.history['val_root_mean_squared_error'], label='validation data')
    ax2.axhline(y = min_loss, color=val_plot2[0].get_color(), linestyle='--') 
    x0,x1 = ax2.get_xlim()
    ax2.text(x1, min_loss, '{:.2f}'.format(min_loss), ha='left', va='center')
    ax2.set_title('RMSE loss')
    ax2.set_ylabel('RMSE')
    ax2.set_xlabel('epochs')
    ax2.legend()
    
    ax3.plot(history.history['mean_absolute_error'], label='training data')    
    min_loss = min(history.history['val_mean_absolute_error'])
    val_plot3 = ax3.plot(history.history['val_mean_absolute_error'], label='validation data')
    ax3.axhline(y=min_loss, color=val_plot3[0].get_color(), linestyle='--') 
    x0,x1 = ax3.get_xlim()
    ax3.text(x1, min_loss, "{:.2f}".format(min_loss), ha='left', va='center')
    ax3.set_title('MAE loss')
    ax3.set_ylabel('MAE')
    ax3.set_xlabel('epochs')
    ax3.legend()

In [ ]:
# This has to be repeated multiple times because the output from optimizer prevents proper display of the plot
history = optimizer.fit_model(0)

In [ ]:
plot_loss(history)

In [ ]:
history = optimizer.fit_model(1)

In [ ]:
plot_loss(history)

In [ ]:
history = optimizer.fit_model(2)

In [ ]:
plot_loss(history)

In [ ]:
history = optimizer.fit_model(3)

In [ ]:
plot_loss(history)

In [ ]:
history = optimizer.fit_model(4)

In [ ]:
plot_loss(history)

In [ ]:
for i, p in enumerate(optimizer.optimization_params):
    prediction, rmse, mae = optimizer.predict(i)
    print('\n'.join((
        f'Performance of simple FC neural network regressor #{i} ({p}):',
        f'  RMSE: {rmse}',
        f'  MAE: {mae}.',
    )))